# Aula 11: Iteradores e funções customizadas

### Luca Mizrahi

In [1]:
!nvidia-smi

Wed Sep 18 17:03:57 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

#### *Atividade*

In [23]:
%%writefile stocks.cu
#include <thrust/device_vector.h>   // Inclui a biblioteca Thrust para vetores na GPU
#include <thrust/host_vector.h>     // Inclui a biblioteca Thrust para vetores na CPU
#include <thrust/transform.h>       // Biblioteca para operações de transformação
#include <thrust/functional.h>      // Para operadores aritméticos
#include <thrust/count.h>           // Para contagem condicional
#include <thrust/replace.h>         // Para substituir valores baseados em condição
#include <thrust/reduce.h>          // Para somar os valores dos vetores
#include <thrust/iterator/constant_iterator.h>  // Para o iterador constante
#include <iostream>                 // Biblioteca padrão para entrada e saída de dados
#include <chrono>                   // Biblioteca para medir o tempo de execução do código
#include <cuda_runtime.h>           // Biblioteca CUDA para medir uso de memória
#include <fstream>                  // Biblioteca para ler arquivos

using namespace std;

// Functor para calcular a diferença entre preços consecutivos
struct DiferencaDiaria {
    __host__ __device__ double operator()(const double& x, const double& y) const {
        return y - x;
    }
};

// Functor para contar aumentos de preço
struct PrecoSubiu {
    __host__ __device__ bool operator()(const double& x) const {
        return x > 0;
    }
};

// Functor para calcular a diferença ao quadrado para variância
struct QuadradoDiferenca {
    __host__ __device__ double operator()(const double& x, const double& media) const {
        double diff = x - media;
        return diff * diff;
    }
};

// Função para obter o uso de memória na GPU
void getMemoryUsage() {
    size_t free_byte;
    size_t total_byte;
    cudaMemGetInfo(&free_byte, &total_byte);

    double free_mb = (double)free_byte / 1024.0 / 1024.0;
    double total_mb = (double)total_byte / 1024.0 / 1024.0;
    double used_mb = total_mb - free_mb;

    cout << "Memória total da GPU: " << total_mb << " MB" << endl;
    cout << "Memória usada da GPU: " << used_mb << " MB" << endl;
    cout << "Memória livre da GPU: " << free_mb << " MB" << endl;
}

int main() {
    // Abrir o arquivo stocks-google.txt para leitura
    ifstream inputFile("stocks-google.txt");

    // Verificar se o arquivo foi aberto corretamente
    if (!inputFile) {
        cerr << "Erro ao abrir o arquivo." << endl;
        return 1;
    }

    // Variáveis para leitura dos preços e contagem de linhas
    double value;
    vector<double> prices;  // Usamos um vetor dinâmico para armazenar os preços

    // Ler os preços do arquivo
    while (inputFile >> value) {
        prices.push_back(value);  // Armazenar cada valor lido
    }

    // Fechar o arquivo após a leitura
    inputFile.close();

    // Determinar o número total de dias (linhas) lidos do arquivo
    int n = prices.size();
    cout << "Número total de dias: " << n << " dias" << endl;

    // Medir o tempo total de execução
    auto exec_start = std::chrono::steady_clock::now();

    // Marca o início da medição do tempo de cópia dos dados da CPU para a GPU
    auto copia_i = std::chrono::steady_clock::now();

    // Criar um vetor na CPU (host_vector) a partir do vetor prices
    thrust::host_vector<double> host(prices.begin(), prices.end());

    // Criar um vetor na GPU (device_vector) copiando os dados do vetor host
    thrust::device_vector<double> dev(host);

    // Marca o fim da medição do tempo de cópia dos dados
    auto copia_f = std::chrono::steady_clock::now();
    std::chrono::duration<double> diff = copia_f - copia_i;
    cout << "Tempo de CÓPIA (em segundos)  " << diff.count() << endl;

    // Exibe o uso de memória da GPU
    getMemoryUsage();

    // Calcula as diferenças diárias
    thrust::device_vector<double> ganho_diario(n - 1);
    auto calc_diff_i = std::chrono::steady_clock::now();
    thrust::transform(dev.begin(), dev.end() - 1, dev.begin() + 1, ganho_diario.begin(), DiferencaDiaria());
    auto calc_diff_f = std::chrono::steady_clock::now();
    diff = calc_diff_f - calc_diff_i;
    cout << "Tempo de CÁLCULO DAS DIFERENÇAS (em segundos): " << diff.count() << endl;

    // Contar os dias com aumento de preço
    int dias_com_aumento = thrust::count_if(ganho_diario.begin(), ganho_diario.end(), PrecoSubiu());
    cout << "Dias com aumento de preço: " << dias_com_aumento << endl;

    // Substituir valores negativos por zero
    thrust::replace_if(ganho_diario.begin(), ganho_diario.end(), thrust::placeholders::_1 < 0, 0.0);

    // Somar os aumentos restantes
    double soma_aumentos = thrust::reduce(ganho_diario.begin(), ganho_diario.end(), 0.0);

    // Calcular aumento médio
    if (dias_com_aumento > 0) {
        double aumento_medio = soma_aumentos / dias_com_aumento;
        cout << "Aumento médio: " << aumento_medio << endl;
    } else {
        cout << "Nenhum aumento registrado." << endl;
    }

    // Calcular a média dos preços
    double soma_precos = thrust::reduce(dev.begin(), dev.end(), 0.0);
    double media = soma_precos / n;

    // Usar thrust::constant_iterator para iterar sobre a média
    thrust::constant_iterator<double> media_iter(media);

    // Calcular a variância
    thrust::device_vector<double> variancias(n);
    thrust::transform(dev.begin(), dev.end(), media_iter, variancias.begin(), QuadradoDiferenca());

    // Somar os valores da variância
    double soma_variancias = thrust::reduce(variancias.begin(), variancias.end(), 0.0);
    double variancia = soma_variancias / n;

    cout << "A média dos preços: " << media << endl;
    cout << "Variância dos preços: " << variancia << endl;

    // Exibe o tempo total de execução
    auto exec_end = std::chrono::steady_clock::now();
    diff = exec_end - exec_start;
    cout << "Tempo TOTAL de EXECUÇÃO (em segundos): " << diff.count() << endl;

    return 0;
}

Overwriting stocks.cu


In [24]:
!nvcc -arch=sm_75 -std=c++14 stocks.cu -o stocks

In [25]:
!./stocks

Número total de dias: 3112 dias
Tempo de CÓPIA (em segundos)  0.185683
Memória total da GPU: 15102.1 MB
Memória usada da GPU: 105 MB
Memória livre da GPU: 14997.1 MB
Tempo de CÁLCULO DAS DIFERENÇAS (em segundos): 2.3759e-05
Dias com aumento de preço: 3056
Aumento médio: 3.15251
A média dos preços: 1580.08
Variância dos preços: 123792
Tempo TOTAL de EXECUÇÃO (em segundos): 0.186937


Observando a saída do código, é possível perceber que o uso de memória foi muito limitado, em relação ao total de memória disponível da GPU. Além disso, do tempo total de execução de 0.2 segundos, é possível observar que a vasta maioria desse tempo foi utilizado na leitura e cópia dos dados da entrada, sendo que o tempo utilizado para os cálculos utilizando os preços das ações foi praticamente irrelevante.

#### *Considere como iteradores dinâmicos e funções de transformação ajudam a otimizar o processamento.*

1. **Iteradores Dinâmicos**: Em vez de armazenar dados desnecessários em memória, iteradores dinâmicos calculam resultados apenas quando são acessados. Isso evita a alocação de memória para armazenar intermediários, como médias ou valores de transformação, reduzindo o consumo de memória e tornando o processo mais eficiente. No caso do **`thrust::transform`**, os valores são calculados no momento da necessidade, sem criar cópias extras no dispositivo.

2. **Funções de Transformação**: A função `thrust::transform` aplica operações entre elementos de dois vetores diretamente na GPU, otimizando o processamento paralelo. Isso é especialmente eficiente em grandes datasets, como o processamento de preços de ações, onde podemos calcular a diferença entre dias consecutivos diretamente na GPU, reduzindo significativamente o tempo de execução. Além disso, o uso de `thrust::reduce` para somar e `thrust::replace_if` para filtrar os dados diretamente na GPU evita transferências de memória entre CPU e GPU, acelerando ainda mais o processo.

Essas abordagens permitem que o processamento seja escalável para grandes volumes de dados sem sobrecarregar a GPU com alocações desnecessárias de memória.